<a href="https://colab.research.google.com/github/lfiathan/testing-deployement-model/blob/main/Testing_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import json

In [3]:
# ...existing code...
# Replace Colab / Kaggle download cell with a local-friendly check + move into repo data/
from pathlib import Path
import shutil
import zipfile
import os

# notebook is in notebook/, repo root is parent
repo_root = Path.cwd().parent
data_dir = repo_root / "data"
data_dir.mkdir(parents=True, exist_ok=True)

# look for the dataset in common locations (including Colab /content or repo root)
candidates = [
    Path.cwd(),                 # notebook/
    repo_root,                  # repo root
    Path.home(),                # home
    Path("/content"),           # colab
]
filenames = ["yelp_labelled.txt", "sentiment-labelled-sentences-data-set.zip", "sentiment labelled sentences"]

found = None
for base in candidates:
    for name in filenames:
        p = base / name
        if p.exists():
            found = p
            break
    if found:
        break

# fallback: search recursively for the file name
if not found:
    matches = list(repo_root.rglob("yelp_labelled.txt")) + list(Path.cwd().rglob("yelp_labelled.txt")) + list(Path("/content").rglob("yelp_labelled.txt")) if Path("/content").exists() else []
    if matches:
        found = matches[0]

if not found:
    raise FileNotFoundError(
        "Dataset not found locally. Place 'yelp_labelled.txt' (or the extracted folder) into the repository 'data/' folder."
    )

# move / extract to data_dir
target_txt = data_dir / "yelp_labelled.txt"
if found.is_dir():
    # look inside folder for the file
    inside = next(found.rglob("yelp_labelled.txt"), None)
    if inside:
        shutil.copy2(inside, target_txt)
        print(f"Copied {inside} -> {target_txt}")
    else:
        raise FileNotFoundError(f"No yelp_labelled.txt inside {found}")
elif found.suffix == ".zip":
    # extract zip into data_dir
    with zipfile.ZipFile(found, "r") as z:
        z.extractall(data_dir)
    # try to locate the text file after extraction
    extracted = next(data_dir.rglob("yelp_labelled.txt"), None)
    if extracted:
        if extracted.resolve() != target_txt.resolve():
            shutil.move(str(extracted), str(target_txt))
        print(f"Extracted and moved to {target_txt}")
    else:
        raise FileNotFoundError("Zip extracted but yelp_labelled.txt not found inside the archive.")
else:
    # plain file -> copy
    shutil.copy2(found, target_txt)
    print(f"Copied {found} -> {target_txt}")

# Load dataset from data_dir (now guaranteed present)
dataset_path = target_txt
if not dataset_path.exists():
    raise FileNotFoundError(f"{dataset_path} not found after move/extract.")

df = pd.read_csv(dataset_path, names=['sentence', 'label'], sep='\t')
print("Loaded dataset:", dataset_path)
# ...existing code...

Copied /Users/macm4/repositories/tesing-deployment-model/notebook/sentiment labelled sentences/yelp_labelled.txt -> /Users/macm4/repositories/tesing-deployment-model/data/yelp_labelled.txt
Loaded dataset: /Users/macm4/repositories/tesing-deployment-model/data/yelp_labelled.txt


In [8]:
df = pd.read_csv('../data/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [10]:
# Mengubah seluruh text kedalam bentuk lowercase
df['sentence'] = df['sentence'].str.lower()

# Menghilangkan stopwords
nltk.download('stopwords') # Download stopwords
stop_word = set(stopwords.words('english'))

df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))

# Melakukan split dataset
sentence = df['sentence'].values
label = df['label'].values

sentence_train, sentence_test, label_train, label_test = train_test_split(sentence, label, test_size=0.2, shuffle=False)

# Membuat tokenisasi
filt = '!"#$%&()*+.,-/:;=?@[]^_`{|}~ ' # Filter untuk menghilangkan symbols

tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)

tokenizer.fit_on_texts(sentence_train)

# Menyimpan word_index kedalam sebuah file json
word_index = tokenizer.word_index
processed_dir = repo_root / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)
word_index_path = processed_dir / "word_index.json"

with open(word_index_path, "w") as fp:
    json.dump(word_index, fp)

# Membuat sequences dan melakukan padding
train_sekuens = tokenizer.texts_to_sequences(sentence_train)
test_sekuens = tokenizer.texts_to_sequences(sentence_test)

train_padded = pad_sequences(train_sekuens,
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

[nltk_data] Downloading package stopwords to /Users/macm4/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Membuat model
model = tf.keras.Sequential([
    Embedding(2000, 20, input_length=20),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
num_epochs = 30
history = model.fit(train_padded, label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, label_test),
                    verbose=1)

Epoch 1/30


/Users/macm4/repositories/tesing-deployment-model/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5500 - loss: 0.6874 - val_accuracy: 0.2400 - val_loss: 0.7511
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5650 - loss: 0.6820 - val_accuracy: 0.2400 - val_loss: 0.7856
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5650 - loss: 0.6766 - val_accuracy: 0.2400 - val_loss: 0.7724
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5725 - loss: 0.6659 - val_accuracy: 0.2400 - val_loss: 0.7906
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6212 - loss: 0.6404 - val_accuracy: 0.2550 - val_loss: 0.7743
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7412 - loss: 0.5747 - val_accuracy: 0.3600 - val_loss: 0.7388
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8562 - loss: 0.4592 - val_accuracy: 0.7200 - val_loss: 0.5852
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8813 - loss: 0.3481 - val_accuracy: 0.6050 - val_loss: 0.6508
Epo

In [13]:
model.save(processed_dir / "model.h5")

In [18]:
import subprocess
import tensorflow as _tf

model_h5 = processed_dir / "model.h5"
saved_model_dir = processed_dir / "saved_model"
tfjs_out_dir = processed_dir / "tfjs_model"

# ensure `model` is in memory or load model.h5
if 'model' not in globals():
    if model_h5.exists():
        model = _tf.keras.models.load_model(str(model_h5))
    else:
        raise FileNotFoundError(f"No trained model object or {model_h5} found")

# 1) Try to save as SavedModel using tf.saved_model.save (robust)
try:
    _tf.saved_model.save(model, str(saved_model_dir))
    print(f"Saved SavedModel -> {saved_model_dir}")
    use_saved_model = True
except Exception as e:
    print("tf.saved_model.save failed:", e)
    use_saved_model = False

# 2) Ensure we have a full Keras H5 (not weights-only)
if not model_h5.exists() or use_saved_model is False:
    try:
        # force saving the full model to H5 (structure + weights)
        model.save(str(model_h5))
        print(f"Saved full Keras H5 -> {model_h5}")
    except Exception as e:
        print("Failed to save full H5:", e)
        # try fallback to Keras native format
        fallback = processed_dir / "model.keras"
        model.save(str(fallback))
        model_h5 = fallback
        print(f"Saved Keras native format -> {fallback}")

# 3) Run tensorflowjs_converter, prefer SavedModel if available
converter = shutil.which("tensorflowjs_converter")
if not converter:
    raise SystemExit("tensorflowjs_converter not found. Install: pip install tensorflowjs")

if saved_model_dir.exists():
    cmd = [converter, "--input_format", "tf_saved_model", str(saved_model_dir), str(tfjs_out_dir)]
else:
    cmd = [converter, "--input_format", "keras", str(model_h5), str(tfjs_out_dir)]

print("Running:", " ".join(cmd))
try:
    subprocess.run(cmd, check=True)
    print("TFJS model written to", tfjs_out_dir)
except subprocess.CalledProcessError as e:
    print("tensorflowjs_converter failed:", e)
    print("If you still see 'failed to lookup keras version' the H5 may be weights-only. Make sure you saved the full model object with model.save(...) and retry.")
# ...existing code...

INFO:tensorflow:Assets written to: /Users/macm4/repositories/tesing-deployment-model/data/processed/saved_model/assets


INFO:tensorflow:Assets written to: /Users/macm4/repositories/tesing-deployment-model/data/processed/saved_model/assets


Saved SavedModel -> /Users/macm4/repositories/tesing-deployment-model/data/processed/saved_model
Running: /Users/macm4/repositories/tesing-deployment-model/.venv/bin/tensorflowjs_converter --input_format tf_saved_model /Users/macm4/repositories/tesing-deployment-model/data/processed/saved_model /Users/macm4/repositories/tesing-deployment-model/data/processed/tfjs_model


🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
/Users/macm4/repositories/tesing-deployment-model/.venv/lib/python3.10/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
I0000 00:00:1763448378.888489 1431866 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1763448378.888636 1431866 single_machine.cc:374] Starting new session


TFJS model written to /Users/macm4/repositories/tesing-deployment-model/data/processed/tfjs_model
